In [3]:
%matplotlib inline
import scipy.io as scio
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import netCDF4 as nc
import numpy as np
from Load_data import Data_from_nc
import xarray as xr
import math
import gc

data_land_mask = Data_from_nc('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/surfdata_0.9x1.25_hist_78pfts_CMIP6_simyr1850_c201126.nc')
data_mask = data_land_mask.load_variable('PFTDATA_MASK')
data_mask = data_mask[29:, :]

#create func to compute PR
def comp_PR(ctl,scen,q):
    #first get quantile from ctl
    qctl=ctl.quantile(q,dim='time')
    
    del ctl
    gc.collect()
    

    #get subsample scenarios
    qscen=xr.where(scen>qctl,1,0)
    pscen=qscen.sum('time')/len(qscen.time)
    
    del qscen
    gc.collect()

    #compute PR
    PR_scen=pscen/(1-q)
    PR_scen = np.where(data_mask==0, np.nan, PR_scen)
    PR_scen = np.where(PR_scen==0, np.nan, PR_scen)
    
    return PR_scen

def get_data_from_mat(file, variable):
    var_dict = scio.loadmat(file)
    var = var_dict[variable]
    var = var[:, 29:, :]
    return var

data_surface = Data_from_nc(\
                            '/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/' + \
                            'surfdata_irrigation_method.nc'\
                           )   #load the data

data_irrigation_method = data_surface.load_variable('irrigation_method')

data_lon = data_surface.load_variable('LONGXY')
data_lon = data_lon[0,:]
data_lat = data_surface.load_variable('LATIXY')
data_lat = data_lat[29:,0]
data_lat = data_lat[:]
data_time = np.array(range(262800))

def get_data_for_year(metric, period, exp, ens):
    data_out = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/CNRM-CM6-1_hist-' + exp + '_' + ens + '/' +\
                                             metric + '_' + period + '_3_hourly_IRR01_0.9x1.25.mat', metric)
    data_out = np.array(data_out)
    return data_out


def nparray_to_xarray_for_this_study(np_array):
    data_xrray = xr.DataArray(np_array, coords={'time': data_time, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    return data_xrray
    
def calcu_PR_between_two_array(data_ref, data_new, perc):
    data_xr_ref = xr.DataArray(data_ref, coords={'time': data_time, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    data_xr_new = xr.DataArray(data_new, coords={'time': data_time, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    PR_result = comp_PR(data_xr_ref,data_xr_new, perc)
    return PR_result


def calcu_3_PRs(metric):
    data_IRR01_1901_1930 = get_data_for_year(metric, '1901', 'irr', 'r1i1p1f2')
    data_NOI01_1901_1930 = get_data_for_year(metric, '1901', 'noirr', 'r1i1p1f2')
    
    data_IRR02_1901_1930 = get_data_for_year(metric, '1901', 'irr', 'r2i1p1f2')
    data_NOI02_1901_1930 = get_data_for_year(metric, '1901', 'noirr', 'r2i1p1f2')
    
    data_IRR03_1901_1930 = get_data_for_year(metric, '1901', 'irr', 'r3i1p1f2')
    data_NOI03_1901_1930 = get_data_for_year(metric, '1901', 'noirr', 'r3i1p1f2')
    
    data_IRR04_1901_1930 = get_data_for_year(metric, '1901', 'irr', 'r4i1p1f2')
    data_NOI04_1901_1930 = get_data_for_year(metric, '1901', 'noirr', 'r4i1p1f2')
    
    data_IRR05_1901_1930 = get_data_for_year(metric, '1901', 'irr', 'r5i1p1f2')
    data_NOI05_1901_1930 = get_data_for_year(metric, '1901', 'noirr', 'r5i1p1f2')
    
    for period in range(1902, 1931):
        data_IRR01_year = get_data_for_year(metric, str(period), 'irr', 'r1i1p1f2')
        data_NOI01_year = get_data_for_year(metric, str(period), 'noirr', 'r1i1p1f2')
        
        data_IRR02_year = get_data_for_year(metric, str(period), 'irr', 'r2i1p1f2')
        data_NOI02_year = get_data_for_year(metric, str(period), 'noirr', 'r2i1p1f2')
        
        data_IRR03_year = get_data_for_year(metric, str(period), 'irr', 'r3i1p1f2')
        data_NOI03_year = get_data_for_year(metric, str(period), 'noirr', 'r3i1p1f2')
        
        data_IRR04_year = get_data_for_year(metric, str(period), 'irr', 'r4i1p1f2')
        data_NOI04_year = get_data_for_year(metric, str(period), 'noirr', 'r4i1p1f2')
        
        data_IRR05_year = get_data_for_year(metric, str(period), 'irr', 'r5i1p1f2')
        data_NOI05_year = get_data_for_year(metric, str(period), 'noirr', 'r5i1p1f2')
        
        data_IRR01_1901_1930 = np.concatenate((data_IRR01_1901_1930, data_IRR01_year), axis = 0)
        data_NOI01_1901_1930 = np.concatenate((data_NOI01_1901_1930, data_NOI01_year), axis = 0)
        
        data_IRR02_1901_1930 = np.concatenate((data_IRR02_1901_1930, data_IRR02_year), axis = 0)
        data_NOI02_1901_1930 = np.concatenate((data_NOI02_1901_1930, data_NOI02_year), axis = 0)
        
        data_IRR03_1901_1930 = np.concatenate((data_IRR03_1901_1930, data_IRR03_year), axis = 0)
        data_NOI03_1901_1930 = np.concatenate((data_NOI03_1901_1930, data_NOI03_year), axis = 0)
        
        data_IRR04_1901_1930 = np.concatenate((data_IRR04_1901_1930, data_IRR04_year), axis = 0)
        data_NOI04_1901_1930 = np.concatenate((data_NOI04_1901_1930, data_NOI04_year), axis = 0)
        
        data_IRR05_1901_1930 = np.concatenate((data_IRR05_1901_1930, data_IRR05_year), axis = 0)
        data_NOI05_1901_1930 = np.concatenate((data_NOI05_1901_1930, data_NOI05_year), axis = 0)
    
    data_IRR_1901_1930 = np.concatenate((data_IRR01_1901_1930,
                                         data_IRR02_1901_1930,
                                         data_IRR03_1901_1930,
                                         data_IRR04_1901_1930,
                                         data_IRR05_1901_1930,
                                         ), axis = 0)
    
    del data_IRR01_1901_1930,data_IRR02_1901_1930,data_IRR03_1901_1930,data_IRR04_1901_1930,data_IRR05_1901_1930
    gc.collect()
    data_time = np.array(range(data_IRR_1901_1930.shape[0]))
    data_xr_ref = xr.DataArray(data_IRR_1901_1930, coords={'time': data_time, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    qctl_IRR_1901_1930_99 = data_xr_ref.quantile(0.99,dim='time')
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PercentileValue_IRR_1901_1930_" + \
                     metric + "_99.0%.mat", {'qctl_IRR_1901_1930_99': qctl_IRR_1901_1930_99})
    
    qctl_IRR_1901_1930_995 = data_xr_ref.quantile(0.995,dim='time')
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PercentileValue_IRR_1901_1930_" + \
                     metric + "_99.5%.mat", {'qctl_IRR_1901_1930_995': qctl_IRR_1901_1930_995})
    
    qctl_IRR_1901_1930_999 = data_xr_ref.quantile(0.999,dim='time')
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PercentileValue_IRR_1901_1930_" + \
                     metric + "_99.9%.mat", {'qctl_IRR_1901_1930_999': qctl_IRR_1901_1930_999})
    
    data_NOI_1901_1930 = np.concatenate((data_NOI01_1901_1930,
                                         data_NOI02_1901_1930,
                                         data_NOI03_1901_1930,
                                         data_NOI04_1901_1930,
                                         data_NOI05_1901_1930,
                                         ), axis = 0)
    
    del data_NOI01_1901_1930,data_NOI02_1901_1930,data_NOI03_1901_1930,data_NOI04_1901_1930,data_NOI05_1901_1930
    gc.collect()
    
    data_xr_ref = xr.DataArray(data_NOI_1901_1930, coords={'time': data_time, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    qctl_NOI_1901_1930_99 = data_xr_ref.quantile(0.99,dim='time')
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PercentileValue_NOI_1901_1930_" + \
                     metric + "_99.0%.mat", {'qctl_NOI_1901_1930_99': qctl_NOI_1901_1930_99})
    qctl_NOI_1901_1930_995 = data_xr_ref.quantile(0.995,dim='time')
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PercentileValue_NOI_1901_1930_" + \
                     metric + "_99.5%.mat", {'qctl_NOI_1901_1930_995': qctl_NOI_1901_1930_995})
    qctl_NOI_1901_1930_999 = data_xr_ref.quantile(0.999,dim='time')
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PercentileValue_NOI_1901_1930_" + \
                     metric + "_99.9%.mat", {'qctl_NOI_1901_1930_999': qctl_NOI_1901_1930_999})
    
    data_IRR01_1985_2014 = get_data_for_year(metric, '1985', 'irr', 'r1i1p1f2')
    data_NOI01_1985_2014 = get_data_for_year(metric, '1985', 'noirr', 'r1i1p1f2')
    
    data_IRR02_1985_2014 = get_data_for_year(metric, '1985', 'irr', 'r2i1p1f2')
    data_NOI02_1985_2014 = get_data_for_year(metric, '1985', 'noirr', 'r2i1p1f2')
    
    data_IRR03_1985_2014 = get_data_for_year(metric, '1985', 'irr', 'r3i1p1f2')
    data_NOI03_1985_2014 = get_data_for_year(metric, '1985', 'noirr', 'r3i1p1f2')
    
    data_IRR04_1985_2014 = get_data_for_year(metric, '1985', 'irr', 'r4i1p1f2')
    data_NOI04_1985_2014 = get_data_for_year(metric, '1985', 'noirr', 'r4i1p1f2')
    
    data_IRR05_1985_2014 = get_data_for_year(metric, '1985', 'irr', 'r5i1p1f2')
    data_NOI05_1985_2014 = get_data_for_year(metric, '1985', 'noirr', 'r5i1p1f2')
    
    for period in range(1986, 2015):
        data_IRR01_year = get_data_for_year(metric, str(period), 'irr', 'r1i1p1f2')
        data_NOI01_year = get_data_for_year(metric, str(period), 'noirr', 'r1i1p1f2')
        
        data_IRR02_year = get_data_for_year(metric, str(period), 'irr', 'r2i1p1f2')
        data_NOI02_year = get_data_for_year(metric, str(period), 'noirr', 'r2i1p1f2')
        
        data_IRR03_year = get_data_for_year(metric, str(period), 'irr', 'r3i1p1f2')
        data_NOI03_year = get_data_for_year(metric, str(period), 'noirr', 'r3i1p1f2')
        
        data_IRR04_year = get_data_for_year(metric, str(period), 'irr', 'r4i1p1f2')
        data_NOI04_year = get_data_for_year(metric, str(period), 'noirr', 'r4i1p1f2')
        
        data_IRR05_year = get_data_for_year(metric, str(period), 'irr', 'r5i1p1f2')
        data_NOI05_year = get_data_for_year(metric, str(period), 'noirr', 'r5i1p1f2')
        
        data_IRR01_1985_2014 = np.concatenate((data_IRR01_1985_2014, data_IRR01_year), axis = 0)
        data_NOI01_1985_2014 = np.concatenate((data_NOI01_1985_2014, data_NOI01_year), axis = 0)
        
        data_IRR02_1985_2014 = np.concatenate((data_IRR02_1985_2014, data_IRR02_year), axis = 0)
        data_NOI02_1985_2014 = np.concatenate((data_NOI02_1985_2014, data_NOI02_year), axis = 0)
        
        data_IRR03_1985_2014 = np.concatenate((data_IRR03_1985_2014, data_IRR03_year), axis = 0)
        data_NOI03_1985_2014 = np.concatenate((data_NOI03_1985_2014, data_NOI03_year), axis = 0)
        
        data_IRR04_1985_2014 = np.concatenate((data_IRR04_1985_2014, data_IRR04_year), axis = 0)
        data_NOI04_1985_2014 = np.concatenate((data_NOI04_1985_2014, data_NOI04_year), axis = 0)
        
        data_IRR05_1985_2014 = np.concatenate((data_IRR05_1985_2014, data_IRR05_year), axis = 0)
        data_NOI05_1985_2014 = np.concatenate((data_NOI05_1985_2014, data_NOI05_year), axis = 0)
        
    data_IRR_1985_2014 = np.concatenate((data_IRR01_1985_2014,
                                         data_IRR02_1985_2014,
                                         data_IRR03_1985_2014,
                                         data_IRR04_1985_2014,
                                         data_IRR05_1985_2014,
                                         ), axis = 0)
    
    del data_IRR01_1985_2014,data_IRR02_1985_2014,data_IRR03_1985_2014,data_IRR04_1985_2014,data_IRR05_1985_2014
    gc.collect()
    
    data_xr_ref = xr.DataArray(data_IRR_1985_2014, coords={'time': data_time, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    
    qctl_IRR_1985_2014_99 = data_xr_ref.quantile(0.99,dim='time')
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PercentileValue_IRR_1985_2014_" + \
                     metric + "_99.0%.mat", {'qctl_IRR_1985_2014_99': qctl_IRR_1985_2014_99})
    qctl_IRR_1985_2014_995 = data_xr_ref.quantile(0.995,dim='time')
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PercentileValue_IRR_1985_2014_" + \
                     metric + "_99.5%.mat", {'qctl_IRR_1985_2014_995': qctl_IRR_1985_2014_995})
    qctl_IRR_1985_2014_999 = data_xr_ref.quantile(0.999,dim='time')
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PercentileValue_IRR_1985_2014_" + \
                     metric + "_99.9%.mat", {'qctl_IRR_1985_2014_999': qctl_IRR_1985_2014_999})
    
    data_NOI_1985_2014 = np.concatenate((data_NOI01_1985_2014,
                                         data_NOI02_1985_2014,
                                         data_NOI03_1985_2014,
                                         data_NOI04_1985_2014,
                                         data_NOI05_1985_2014,
                                         ), axis = 0)
    
    del data_NOI01_1985_2014,data_NOI02_1985_2014,data_NOI03_1985_2014,data_NOI04_1985_2014,data_NOI05_1985_2014
    gc.collect()
    
    data_xr_ref = xr.DataArray(data_NOI_1985_2014, coords={'time': data_time, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    qctl_NOI_1985_2014_99 = data_xr_ref.quantile(0.99,dim='time')
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PercentileValue_NOI_1985_2014_" + \
                     metric + "_99.0%.mat", {'qctl_NOI_1985_2014_99': qctl_NOI_1985_2014_99})
    qctl_NOI_1985_2014_995 = data_xr_ref.quantile(0.995,dim='time')
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PercentileValue_NOI_1985_2014_" + \
                     metric + "_99.5%.mat", {'qctl_NOI_1985_2014_995': qctl_NOI_1985_2014_995})
    qctl_NOI_1985_2014_999 = data_xr_ref.quantile(0.999,dim='time')
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PercentileValue_NOI_1985_2014_" + \
                     metric + "_99.9%.mat", {'qctl_NOI_1985_2014_999': qctl_NOI_1985_2014_999})
    
    
    
    return qctl_NOI_1901_1930_99, qctl_NOI_1901_1930_995, qctl_NOI_1901_1930_999, qctl_IRR_1901_1930_99, qctl_IRR_1901_1930_995, qctl_IRR_1901_1930_999, qctl_NOI_1985_2014_99, qctl_NOI_1985_2014_995, qctl_NOI_1985_2014_999, qctl_IRR_1985_2014_99, qctl_IRR_1985_2014_995, qctl_IRR_1985_2014_999
    
def plot_PR_single(ax, data_xarray, title, metric, perc):
    cmap='bwr'
    vmin=1/4
    vmax=4
    alpha=0.05
    levels = [-4, -2, -1, -0.2, 0.2, 1, 2, 4]
    im = data_xarray.plot(ax=ax, cmap=cmap,vmin=vmin,vmax=vmax,levels=levels, extend='both', add_colorbar=False, add_labels=False)
    cb = plt.colorbar(im,fraction= 0.3, pad= 0.04, extend='both', orientation='horizontal')
    cb.ax.tick_params(labelsize=12)
    cb.set_ticks([-4, -2, -1, -0.2, 0.2, 1, 2, 4])
    cb.set_ticklabels(["-4", "-2", "-1", "-0.2", "+0.2", "+1", "+2", "+4"])
    ax.coastlines(color='dimgray', linewidth=0.5)
    ax.add_feature(cfeature.OCEAN, color='lightgrey')
    ax.set_title(title, loc='right', fontsize=14)
    ax.set_title(metric + '_' + perc, loc='left', fontsize=14)
    ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=0.1)
    
    
    
def just_do_all_even_more(metric, varname):
    qctl_NOI_1901_1930_99, qctl_NOI_1901_1930_995, qctl_NOI_1901_1930_999, qctl_IRR_1901_1930_99, qctl_IRR_1901_1930_995, qctl_IRR_1901_1930_999, qctl_NOI_1985_2014_99, qctl_NOI_1985_2014_995, qctl_NOI_1985_2014_999, qctl_IRR_1985_2014_99, qctl_IRR_1985_2014_995, qctl_IRR_1985_2014_999 = calcu_3_PRs(metric)
    
    fig = plt.figure(figsize=(18,12),dpi=300)
    proj=ccrs.PlateCarree()
    
    
    data_xr_1 = xr.DataArray(qctl_IRR_1985_2014_99-qctl_IRR_1901_1930_99, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(331, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_1, 'all forcings', varname, '99.0%')
    ax.text(0.01, 0.92, 'a', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')

    data_xr_2 = xr.DataArray(qctl_NOI_1985_2014_99-qctl_NOI_1901_1930_99, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(334, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_2, 'other forcings', varname, '99.0%')
    ax.text(0.01, 0.92, 'd', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')
    
    data_xr_3 = xr.DataArray(qctl_IRR_1985_2014_99-qctl_NOI_1985_2014_99, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(337, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_3, 'irrigation expansion', varname, '99.0%')
    ax.text(0.01, 0.92, 'g', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')
    
    
    data_xr_4 = xr.DataArray(qctl_IRR_1985_2014_995-qctl_IRR_1901_1930_995, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(332, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_4, 'all forcings', varname, '99.5%')
    ax.text(0.01, 0.92, 'b', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')
    

    data_xr_5 = xr.DataArray(qctl_NOI_1985_2014_995-qctl_NOI_1901_1930_995, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(335, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_5, 'other forcings', varname, '99.5%')
    ax.text(0.01, 0.92, 'e', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')

    data_xr_6 = xr.DataArray(qctl_IRR_1985_2014_995-qctl_NOI_1985_2014_995, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(338, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_6, 'irrigation expansion', varname, '99.5%')
    ax.text(0.01, 0.92, 'h', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')
    
    
    
    data_xr_7 = xr.DataArray(qctl_IRR_1985_2014_999-qctl_IRR_1901_1930_999, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(333, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_7, 'all forcings', varname, '99.9%')
    ax.text(0.01, 0.92, 'c', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')
    

    data_xr_8 = xr.DataArray(qctl_NOI_1985_2014_999-qctl_NOI_1901_1930_999, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(336, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_8, 'other forcings', varname, '99.9%')
    ax.text(0.01, 0.92, 'f', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')

    data_xr_9 = xr.DataArray(qctl_IRR_1985_2014_999-qctl_NOI_1985_2014_999, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(339, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_9, 'irrigation expansion', varname, '99.9%')
    ax.text(0.01, 0.92, 'i', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')

In [ ]:
just_do_all_even_more('TSA', 'CNRM-CM6 T_2m')
just_do_all_even_more('DI', 'CNRM-CM6 DI')
just_do_all_even_more('HUMIDEX', 'CNRM-CM6 HU')
just_do_all_even_more('THIC', 'CNRM-CM6 THIC')
just_do_all_even_more('THIP', 'CNRM-CM6 THIP')
just_do_all_even_more('T_s_65', 'CNRM-CM6 T_s_65')
just_do_all_even_more('T_s_80', 'CNRM-CM6 T_s_80')
just_do_all_even_more('T_w', 'CNRM-CM6 T_w')